In [43]:
import pysrt
import pandas as pd
import moviepy.editor as mp
from glob import glob
import re
import os
import soundfile as sf
from tqdm import tqdm
pd.options.display.max_rows=999
pd.options.display.max_columns=999

In [2]:
subs = pysrt.open('Subtitles.srt')

In [3]:
s_timestamp=[]
e_timestamp=[]
subtitle=[]
for i in range(0,len(subs),1):
    s_timestamp.append(subs[i].start.to_time())
    e_timestamp.append(subs[i].end.to_time())
    subtitle.append(subs[i].text)

In [4]:
df=pd.DataFrame({"s_timestamp":s_timestamp,"e_timestamp":e_timestamp,"subtitle":subtitle})

In [5]:
print(subs[1].start.to_time(),subs[1].end.to_time())

00:00:23.606000 00:00:27.484000


In [6]:
df.head(5)

,s_timestamp,e_timestamp,subtitle
0,00:00:17.142000,00:00:19.226000,♪ THE INK SPOTS: If I Didn't Care
1,00:00:23.606000,00:00:27.484000,♪ If I didn't care
2,00:00:28.737000,00:00:32.990000,♪ More than words can say
3,00:00:34.075000,00:00:38.245000,♪ If I didn't care
4,00:00:39.456000,00:00:43.459000,♪ Would I feel this way?


In [5]:
clip = mp.VideoFileClip("Movie.mp4").subclip(17,20)
clip.audio.write_audiofile("theaudio.mp3")

MoviePy - Writing audio in theaudio.mp3


MoviePy - Done.


In [7]:
def mailabs(root_path, meta_files=None):
    """Normalizes M-AI-Labs meta data files to TTS format"""
    speaker_regex = re.compile("by_book/(male|female)/(?P<speaker_name>[^/]+)/")
    if meta_files is None:
        csv_files = glob(root_path+"/**/metadata.csv", recursive=True)
    else:
        csv_files = meta_files
    # meta_files = [f.strip() for f in meta_files.split(",")]
    items = []
    for csv_file in csv_files:
        txt_file = os.path.join(root_path, csv_file)
        folder = os.path.dirname(txt_file)
        # determine speaker based on folder structure...
        speaker_name_match = speaker_regex.search(txt_file)
        if speaker_name_match is None:
            continue
        speaker_name = speaker_name_match.group("speaker_name")
        print(" | > {}".format(csv_file))
        with open(txt_file, 'r') as ttf:
            for line in ttf:
                cols = line.split('|')
                if meta_files is None:
                    wav_file = os.path.join(folder, 'wavs', cols[0] + '.wav')
                else:
                    wav_file = os.path.join(root_path, folder.replace("metadata.csv", ""), 'wavs', cols[0] + '.wav')
                if os.path.isfile(wav_file):
                    text = cols[1].strip()
                    items.append([text, wav_file, speaker_name])
                else:
                    raise RuntimeError("> File %s is not exist!"%(wav_file))
    return items

In [8]:
root_path=r'D:\en_UK'
mailabs(root_path,meta_files='metadata.csv')

[]

In [9]:
cd D:\en_UK\by_book\female\elizabeth_klett\jane_eyre

D:\en_UK\by_book\female\elizabeth_klett\jane_eyre


In [10]:
metadata=pd.read_csv("metadata.csv",header=None)

In [58]:
metadata.columns=['ID','Text']

In [24]:

text=list(metadata.iloc[:,1].values)

In [11]:
len(str(metadata.iloc[:1,1]))

80

In [27]:
head_text=[]
for line in text:
    head, sep, tail = line.partition('|')
    head_text.append(head)


In [32]:
metadata['Text']=head_text

In [37]:
audio_files=glob(r'D:\en_UK\by_book\female\elizabeth_klett\jane_eyre\wavs'+"/*.wav")

In [57]:
audio_files[0][-24:-4]

'jane_eyre_01_f000001'

In [39]:
metadata.shape

(10057, 2)

In [48]:
path = 'D:\en_UK\by_book\female\elizabeth_klett\jane_eyre\wavs'
audios = []
# total = len(os.listdir(path))
total = len(audio_files)
pbar = tqdm(total = total)
for file in audio_files:
    data, sr = sf.read(file)
    audios.append(data)
    pbar.update(1)
pbar.close()




  0%|                                                                                        | 0/10055 [00:00<?, ?it/s]


  0%|▏                                                                             | 30/10055 [00:00<00:34, 291.45it/s]


  1%|▍                                                                             | 61/10055 [00:00<00:33, 296.31it/s]


  1%|▊                                                                             | 99/10055 [00:00<00:31, 316.83it/s]


  1%|▉                                                                            | 130/10055 [00:00<00:31, 313.65it/s]


  2%|█▏                                                                           | 163/10055 [00:00<00:31, 313.93it/s]


  2%|█▍                                                                           | 194/10055 [00:00<00:31, 311.85it/s]


  2%|█▋                                                                           | 226/10055 [00:00<00:31, 311.48it/s]


  3%|██              

 23%|█████████████████▎                                                          | 2287/10055 [00:13<00:23, 326.86it/s]


 23%|█████████████████▌                                                          | 2321/10055 [00:13<00:23, 328.91it/s]


 23%|█████████████████▊                                                          | 2355/10055 [00:13<00:23, 324.38it/s]


 24%|██████████████████▏                                                         | 2400/10055 [00:14<00:21, 353.12it/s]


 24%|██████████████████▍                                                         | 2441/10055 [00:14<00:20, 368.37it/s]


 25%|██████████████████▋                                                         | 2479/10055 [00:14<00:21, 359.52it/s]


 25%|███████████████████                                                         | 2516/10055 [00:14<00:23, 315.03it/s]


 25%|███████████████████▎                                                        | 2549/10055 [00:14<00:25, 297.14it/s]


 26%|███████████████████

 41%|██████████████████████████████▊                                             | 4081/10055 [00:28<00:45, 132.17it/s]


 41%|██████████████████████████████▉                                             | 4098/10055 [00:28<00:49, 120.54it/s]


 41%|███████████████████████████████▏                                            | 4123/10055 [00:28<00:41, 142.55it/s]


 41%|███████████████████████████████▍                                            | 4153/10055 [00:28<00:34, 168.66it/s]


 42%|███████████████████████████████▌                                            | 4175/10055 [00:28<00:34, 172.60it/s]


 42%|███████████████████████████████▊                                            | 4202/10055 [00:29<00:30, 193.50it/s]


 42%|███████████████████████████████▉                                            | 4230/10055 [00:29<00:27, 212.01it/s]


 42%|████████████████████████████████▏                                           | 4254/10055 [00:29<00:26, 218.46it/s]


 43%|███████████████████

 60%|█████████████████████████████████████████████▋                              | 6047/10055 [00:39<00:16, 240.11it/s]


 60%|█████████████████████████████████████████████▉                              | 6074/10055 [00:39<00:16, 245.10it/s]


 61%|██████████████████████████████████████████████▏                             | 6104/10055 [00:39<00:15, 258.13it/s]


 61%|██████████████████████████████████████████████▎                             | 6131/10055 [00:40<00:15, 252.60it/s]


 61%|██████████████████████████████████████████████▌                             | 6162/10055 [00:40<00:14, 266.72it/s]


 62%|██████████████████████████████████████████████▊                             | 6196/10055 [00:40<00:13, 283.56it/s]


 62%|███████████████████████████████████████████████                             | 6226/10055 [00:40<00:15, 244.69it/s]


 62%|███████████████████████████████████████████████▎                            | 6252/10055 [00:40<00:17, 219.18it/s]


 62%|███████████████████

 77%|██████████████████████████████████████████████████████████▊                 | 7787/10055 [00:49<00:10, 207.48it/s]


 78%|███████████████████████████████████████████████████████████                 | 7809/10055 [00:49<00:11, 195.63it/s]


 78%|███████████████████████████████████████████████████████████▏                | 7830/10055 [00:49<00:12, 184.33it/s]


 78%|███████████████████████████████████████████████████████████▎                | 7852/10055 [00:49<00:11, 192.37it/s]


 78%|███████████████████████████████████████████████████████████▌                | 7875/10055 [00:49<00:10, 201.53it/s]


 79%|███████████████████████████████████████████████████████████▋                | 7896/10055 [00:49<00:11, 194.67it/s]


 79%|███████████████████████████████████████████████████████████▊                | 7916/10055 [00:50<00:15, 142.56it/s]


 79%|████████████████████████████████████████████████████████████▋                | 7933/10055 [00:50<00:25, 83.45it/s]


 79%|███████████████████

 90%|████████████████████████████████████████████████████████████████████▏       | 9018/10055 [01:02<00:03, 264.80it/s]


 90%|████████████████████████████████████████████████████████████████████▍       | 9061/10055 [01:02<00:03, 298.85it/s]


 90%|████████████████████████████████████████████████████████████████████▋       | 9094/10055 [01:03<00:03, 270.70it/s]


 91%|████████████████████████████████████████████████████████████████████▉       | 9124/10055 [01:03<00:03, 239.77it/s]


 91%|█████████████████████████████████████████████████████████████████████▏      | 9151/10055 [01:03<00:03, 231.77it/s]


 91%|█████████████████████████████████████████████████████████████████████▎      | 9176/10055 [01:03<00:03, 234.31it/s]


 92%|█████████████████████████████████████████████████████████████████████▌      | 9201/10055 [01:03<00:03, 227.37it/s]


 92%|█████████████████████████████████████████████████████████████████████▋      | 9225/10055 [01:03<00:03, 222.58it/s]


 92%|███████████████████

In [63]:
audio_id=[]
for name in audio_files:
    id=name[-24:-4]
    audio_id.append(id)
len(audio_id)    

10055

In [65]:
dataset = pd.DataFrame(columns = ['Audio_ID','Audio'])
dataset.Audio = audios
dataset.Audio_ID = audio_id

In [74]:
Final_data=dataset.merge(metadata,left_on='Audio_ID',
    right_on='ID',)

In [83]:
# Final_data=Final_data.drop(['ID'],axis=0)
Final_data.head()

,Audio_ID,Audio,Text
0,jane_eyre_01_f000001,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",Chapter 1 of Jane eyre. This is there librivox...
1,jane_eyre_01_f000002,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",Recording by Elisabeth Klett.
2,jane_eyre_01_f000003,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",Jane eyre by Charlotte Bronte. Chapter 1.
3,jane_eyre_01_f000004,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",There was no possibility of taking a walk that...
4,jane_eyre_01_f000005,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",We had been wandering indeed in the leafless s...
